In [ ]:
import os
import pandas as pd
from imageio import imread
import cv2
from pycocotools import mask as pymask
import matplotlib.pyplot as plt
import numpy as np
from abcmodel.lib.datasets.cvat import Polygon

DIR = '/workspace/pig'
DATA_DIR = os.path.join(DIR, 'data/images_20211117')
# DATA_DIR = os.path.join(DIR, 'data/images_20211125/images/')

In [ ]:
df = pd.read_pickle(os.path.join(DIR, 'data/akiyama_20211202.pkl'))
# df = pd.read_pickle(os.path.join(DATA_DIR, 'df_final.pkl'))
df = df.reset_index()

In [ ]:
# remove_idx = pd.read_csv('../remove_idx.csv', header=None)
# remove_idx = list(remove_idx.values.flatten())

# new_df = df[df.apply(lambda x: x['index']  not in remove_idx, axis=1)]
# new_df.shape

In [ ]:
import copy
def rotate_head_tail(image_local, mask_local, depth_local):
    h,w,_ = image_local.shape
    center = (int(w/2), int(h/2))
    if h > w:
        indices_y, indices_x = np.where(mask_local)
        min_y = np.min(indices_y)
        max_y = np.max(indices_y)
        upper_x = int(np.round(np.mean(indices_x[np.where(indices_y==min_y)])))
        lower_x = int(np.round(np.mean(indices_x[np.where(indices_y==max_y)])))
        upper_side = np.array([min_y, upper_x])
        lower_side = np.array([max_y, lower_x])
        theta = np.arctan((upper_side[0]-lower_side[0])/(upper_side[1]-lower_side[1]))
    else:
        indices_y, indices_x = np.where(mask_local)
        min_x = np.min(indices_x)
        max_x = np.max(indices_x)
        upper_y = int(np.round(np.mean(indices_y[np.where(indices_x==min_x)])))
        lower_y = int(np.round(np.mean(indices_y[np.where(indices_x==max_x)])))
        upper_side = np.array([upper_y, min_x])
        lower_side = np.array([lower_y, max_x])
        theta = np.arctan((upper_side[0]-lower_side[0])/(upper_side[1]-lower_side[1]))

    #  print("side: ", upper_side, lower_side)
    angle = 180 * theta  / np.pi
    # print("angle: ", angle)
    trans = cv2.getRotationMatrix2D(center, angle , scale=1.0)

    # 回転後の画像サイズを計算
    w_rot = int(np.round(h*np.absolute(np.sin(theta))+w*np.absolute(np.cos(theta))))
    h_rot = int(np.round(h*np.absolute(np.cos(theta))+w*np.absolute(np.sin(theta))))
    size_rot = (w_rot, h_rot)

    trans[0][2] = trans[0][2] -w/2 + w_rot/2
    trans[1][2] = trans[1][2] -h/2 + h_rot/2

    rotate_image = copy.deepcopy(image_local)
    rotate_mask = copy.deepcopy(mask_local)
    rotate_depth = copy.deepcopy(depth_local)

    rotate_image = cv2.warpAffine(rotate_image, trans, (w_rot, h_rot))
    rotate_mask = cv2.warpAffine(rotate_mask, trans, (w_rot, h_rot))
    rotate_depth = cv2.warpAffine(rotate_depth, trans, (w_rot, h_rot))

    mask_indices = np.where(rotate_mask)
    xmin = np.min(mask_indices[1])
    ymin = np.min(mask_indices[0])
    xmax = np.max(mask_indices[1])
    ymax = np.max(mask_indices[0])

    rotate_image = rotate_image[ymin:ymax, xmin:xmax]
    rotate_mask = rotate_mask[ymin:ymax, xmin:xmax]
    rotate_depth = rotate_depth[ymin:ymax, xmin:xmax]

    return rotate_image, rotate_mask, rotate_depth

In [ ]:
for i in range(10, 20):
    pig = df.iloc[i]
    print(i, pig.filename)
    image_file = os.path.join(DATA_DIR, pig.filename)
    depth_file = os.path.join(DATA_DIR, pig.filename.replace("rgb", "depth").replace("jpg","png"))
    x0, y0, w, h = pig.bbox
    anno_box = [x0, y0, x0+w, y0+h]
    image = imread(image_file)
    depth = imread(depth_file)
    depth = cv2.resize(depth, (640, 480))
    mask = pymask.decode(pig.polygon._rle)
    mask_local = mask[y0:y0 + h, x0:x0 + w]
    image_local = image[y0:y0 + h, x0:x0 + w]
    depth_local = depth[y0:y0 + h, x0:x0 + w]

    rotate_image, rotate_mask, rotate_depth = \
        rotate_head_tail(image_local, mask_local, depth_local)
    rotate_depth = rotate_depth * rotate_mask
    fig, (ax1,ax2,ax3,ax4,ax5,ax6) = plt.subplots(ncols=6, figsize=(8,4))
    ax1.imshow(image_local)
    ax2.imshow(mask_local)
    ax3.imshow(depth_local)
    ax4.imshow(rotate_image)
    ax5.imshow(rotate_mask)
    ax6.imshow(rotate_depth)
    plt.show()

In [ ]:
print(rotate_mask.shape)
print(rotate_image.shape)

In [ ]:
depth.shape

In [ ]:
for i in range(0, 20):
    pig = df.iloc[i]

    img = imread(os.path.join(DATA_DIR, pig.filename))
    x0, y0, w, h = pig.bbox
    img = img[y0:y0 + h, x0:x0 + w] 

    depth = imread(os.path.join(
        DATA_DIR, pig.filename.replace('rgb', 'depth').replace('.jpg', '.png')))
    depth = cv2.resize(depth, (640, 480))
    depth = depth[y0:y0 + h, x0:x0 + w]
    height = np.where(depth != 0, pig.floor_depth - depth.astype(int), 0)

    mask = pymask.decode(pig.polygon._rle)
    mask = mask[y0:y0 + h, x0:x0 + w]

    print(f'index: {i}, House: {pig["loc"]},  PigID: {pig.UID}')
    fix, ax = plt.subplots(figsize=(8, 3), ncols=3)
    ax[0].imshow(img);
    ax[1].imshow(mask);
    ax[2].imshow(height);
    ax[0].axes.xaxis.set_visible(False)
    ax[0].axes.yaxis.set_visible(False)
    ax[1].axes.xaxis.set_visible(False)
    ax[1].axes.yaxis.set_visible(False)
    ax[2].axes.xaxis.set_visible(False)
    ax[2].axes.yaxis.set_visible(False)
    plt.show()

# len(df)

In [ ]:
pig = df.iloc[136]

img = imread(os.path.join(DATA_DIR, pig.filename))
x0, y0, w, h = pig.bbox
img = img[y0:y0 + h, x0:x0 + w] 

depth = imread(os.path.join(
    DATA_DIR, pig.filename.replace('rgb', 'depth').replace('.jpg', '.png')))
depth = cv2.resize(depth, (640, 480))
depth = depth[y0:y0 + h, x0:x0 + w]
# height = np.where(depth != 0, pig.floor_depth - depth.astype(int), 0)
depth = depth * (pig.floor_depth - depth.astype(int))

mask = pymask.decode(pig.polygon._rle)
mask = mask[y0:y0 + h, x0:x0 + w]

print(f'index: {i}, House: {pig["loc"]},  PigID: {pig.UID}')
fix, ax = plt.subplots(figsize=(8, 3), ncols=3)
ax[0].imshow(img);
ax[1].imshow(mask);
ax[2].imshow(depth);
ax[0].axes.xaxis.set_visible(False)
ax[0].axes.yaxis.set_visible(False)
ax[1].axes.xaxis.set_visible(False)
ax[1].axes.yaxis.set_visible(False)
ax[2].axes.xaxis.set_visible(False)
ax[2].axes.yaxis.set_visible(False)
plt.show()